# 40 – Bootstrap Customers from PaySim (Optional)

In [ ]:
from pyspark.sql import functions as F, Window
RAW = VOLUME_URI_RAW
p = spark.read.csv(RAW + "/paysim/paysim_small.csv", header=True, inferSchema=True)
w = Window.orderBy("nameOrig")
customers = (p.select(F.col("nameOrig").alias("customer_id")).distinct()
               .withColumn("name", F.concat(F.lit("Cust_"), F.row_number().over(w)))
               .withColumn("email", F.concat(F.col("name"), F.lit("@example.com")))
               .withColumn("dob", F.to_date(F.lit("1985-01-01")))
               .withColumn("country", F.lit("IN"))
               .withColumn("national_id", F.concat(F.lit("IN-"), F.monotonically_increasing_id()))
               .withColumn("pep_flag", (F.rand() < 0.02).cast("boolean")))

out = RAW + "/customers/customers.csv"
customers.coalesce(1).write.mode("overwrite").option("header", True).csv(out)
print("Wrote customers to", out)